# Standardisation des données médicales au format OMOP

## 1. Objectif:

Transformer des données médicales hétérogènes en un format standardisé (OMOP).

### Format standardisé OMOP:

Le format standardisé OMOP (Observational Medical Outcomes Partnership) est un modèle de données développé pour faciliter la standardisation et l'intégration des données médicales de diverses sources afin de permettre une analyse comparative efficace et reproductible.

#### Définition de l'OMOP Common Data Model (CDM)

Le OMOP Common Data Model (CDM) est un modèle de données relationnelles conçu pour structurer les données de santé provenant de différentes sources, telles que les dossiers médicaux électroniques, les registres de patients, les données de réclamations d'assurance, et autres. Le CDM permet de :

1. **Standardiser les Formats** : Convertir les données de santé hétérogènes en un format commun et cohérent.
2. **Faciliter l'Analyse** : Permettre des analyses comparatives et reproductibles à grande échelle.
3. **Promouvoir l'Interopérabilité** : Faciliter l'échange et la comparaison de données entre différentes institutions et systèmes.

#### Composants Principaux de l'OMOP CDM

1. **Tables Standardisées** :
   - **Person** : Informations démographiques sur les patients.
   - **Observation Period** : Périodes de temps pendant lesquelles les patients sont observés.
   - **Visit Occurrence** : Informations sur les visites médicales.
   - **Condition Occurrence** : Diagnostiques médicaux des patients.
   - **Drug Exposure** : Informations sur les médicaments prescrits et consommés.
   - **Procedure Occurrence** : Procédures médicales subies par les patients.
   - **Measurement** : Mesures cliniques (ex. résultats de tests de laboratoire).
   - **Observation** : Informations supplémentaires sur les patients qui ne rentrent pas dans les autres tables.

2. **Vocabulaire Standardisé** : Utilisation de terminologies standardisées (ex. SNOMED, RxNorm) pour assurer la cohérence et l'interopérabilité.

3. **Concept ID** : Utilisation d'identifiants de concepts standardisés pour chaque type d'information, ce qui permet de lier les données entre elles de manière cohérente et de faciliter les analyses.

#### Avantages de l'OMOP CDM

- **Analyses Comparatives** : Permet de réaliser des études comparatives à grande échelle entre différentes populations et systèmes de santé.
- **Réplicabilité** : Facilite la réplication des études et des analyses dans différents contextes.
- **Interopérabilité** : Assure que les données peuvent être échangées et comprises entre différents systèmes et institutions.
- **Qualité des Données** : Améliore la qualité et la cohérence des données grâce à des normes strictes de modélisation et de codage.

#### Utilisation et Applications

L'OMOP CDM est utilisé dans de nombreux projets de recherche en santé pour analyser les résultats médicaux, évaluer l'efficacité des traitements, et surveiller la sécurité des médicaments. Il est également au cœur de la communauté OHDSI (Observational Health Data Sciences and Informatics), qui rassemble des chercheurs du monde entier pour collaborer sur des projets de recherche en utilisant des données standardisées.

#### Conclusion

Le modèle de données OMOP est essentiel pour transformer des données de santé hétérogènes en un format standardisé, facilitant ainsi des analyses robustes et comparatives. Il joue un rôle crucial dans l'amélioration de la qualité des soins de santé grâce à une meilleure compréhension des résultats médicaux à travers des données cohérentes et intégrées.

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=2fd17224818d287829e29b458d4bb918200167cf83ae17028897bcd894c01b7b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# 2. Bibliothèque et config

In [2]:

import pandas as pd
import sqlite3
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# config du nb maximal de colonnes affichées (ici inutile car n<=20 lignes elseif use it)
pd.set_option('display.max_columns', None)

# 3. Chargement et Exploration des Données Sources

In [3]:
# data
# bases principales
ir_ben_r = pd.read_csv('ir_ben_r.csv') # Table contenant les informations des assurés
er_prs_f = pd.read_csv('er_prs_f.csv') # Table contenant les informations sur les prestations remboursées

# bases secondaires
T_mcoaaE = pd.read_csv('t_mcoaae.csv',sep=";") # Tables contenant des informations sur les établissements de santé
ir_act_v = pd.read_csv('ir_act_v.csv') # Tables contenant des informations sur les professionnels de santé.
ir_spe_v = pd.read_csv('ir_spe_v.csv') # Tables contenant des informations sur les professionnels de santé.

In [4]:
# Exploration des données
# # Regardons les NA
def Missing_values(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total,percent], axis=1, keys=['Total', 'Pourcentage'])
    #Affiche que les variables avec des na
    print (missing_data[(percent>0)],'\n' )

In [5]:
print("ir_ben_r: \nsize", ir_ben_r.shape)
Missing_values(ir_ben_r)
ir_ben_r.head(3)

ir_ben_r: 
size (20, 6)
Empty DataFrame
Columns: [Total, Pourcentage]
Index: [] 



,NUM_ENQ,ben_sex_cod,ben_nai_ann,ben_nai_moi,ben_res_dpt,ben_res_reg
0,DPXX:00000000000000001X,2,1963,12,75,114
1,DPXX:000000000000002X,1,1971,2,93,114
2,DPXX:000000000000003X,1,1962,12,93,114


In [6]:
print("er_prs_f: \nsize", er_prs_f.shape)
Missing_values(er_prs_f)
er_prs_f.head()

er_prs_f: 
size (20, 8)
             Total  Pourcentage
etb_pre_fin      9         0.45
pse_spe_cod      6         0.30
pse_act_nat      6         0.30 



,id,NUM_ENQ,prs_nat_ref,exe_soi_dtd,exe_soi_dtf,pse_spe_cod,pse_act_nat,etb_pre_fin
0,1,DPXX:000000000000000000000001X,1130,2013-03-04,2013-03-04,1.0,NaN,NaN
1,2,DPXX:000000000000000000000001X,1331,2013-03-05,2013-03-05,6.0,NaN,750300360.0
2,3,DPXX:000000000000000000000001X,3313,2013-03-05,2013-03-05,NaN,50.0,750023772.0
3,4,DPXX:000000000000000000000001X,3125,2013-03-07,2013-03-07,NaN,26.0,NaN
4,5,DPXX:000000000000000000000002X,1130,2013-03-05,2013-07,6.0,26.0,750023772.0


In [7]:
print("T_mcoaaE: \nsize:", T_mcoaaE.shape )
Missing_values(T_mcoaaE)
T_mcoaaE.head()

T_mcoaaE: 
size: (2, 2)
Empty DataFrame
Columns: [Total, Pourcentage]
Index: [] 



,eta_num,soc_rai
0,750300360,l'Hôpital Privé des Peupliers
1,750023772,Pharmacie Plaisance


In [8]:
print("\nir_act_v: \nsize:", ir_act_v.shape)
Missing_values(ir_act_v)
ir_act_v.head()


ir_act_v: 
size: (2, 2)
Empty DataFrame
Columns: [Total, Pourcentage]
Index: [] 



,pfs_act_nat,label
0,26,Kinésithérapeute
1,50,Pharmacien


In [9]:
print("ir_spe_v: \nsize:", ir_spe_v.shape)
Missing_values(ir_spe_v)
ir_spe_v.head()

ir_spe_v: 
size: (2, 2)
Empty DataFrame
Columns: [Total, Pourcentage]
Index: [] 



,pfs_spe_cod,label
0,1,Médecin généraliste
1,6,Radiologue


# 4. Transformation des Données
## 4.1 Table Person

Correspondance des variables:
- person_id ------------ index
- gender_concept_id  --------------- selon la  documentation ATHENA: (8532 pour femme, 8507 pour homme)
- year_of_birth ------------ ir_ben_r["ben__nai_ann"]
- month_of_birth ------------ir_ben_r["ben__nai_moi"]
- person_source_value ------------ ir_ben_r["NUM_ENQ"]
- location_id ------------ ir_ben_r["ben__res_dep"]
- gender_source_value --------------- ir_ben_r["ben_sex_cod"]

In [10]:
# Transformation des données pour la table Person
data = {
    "person_id": list(range(1,21)),
    "gender_concept_id": ir_ben_r['ben_sex_cod'].map({1: 8507, 2: 8532}),  # 8532 is the standard concept ID for 'Female' in OMOP
    "year_of_birth": ir_ben_r["ben_nai_ann"],
    "month_of_birth": ir_ben_r["ben_nai_moi"],
    "person_source_value": ir_ben_r["NUM_ENQ"],
    "location_id": ir_ben_r["ben_res_dpt"],
    "gender_source_value": ir_ben_r['ben_sex_cod']
}

# Créer un DataFrame
person_df = pd.DataFrame(data)

# Sauvegarder le DataFrame en CSV
person_df.to_csv('Person.csv', index=False)

In [11]:
person_df.head(3)

,person_id,gender_concept_id,year_of_birth,month_of_birth,person_source_value,location_id,gender_source_value
0,1,8532,1963,12,DPXX:00000000000000001X,75,2
1,2,8507,1971,2,DPXX:000000000000002X,93,1
2,3,8507,1962,12,DPXX:000000000000003X,93,1


## 4.2 Table Care Site

Correspondance des variables:
- cc_site_id ---------- index
-care_site_name --------- T_mcoaaE["soc_rai"]
- location_id --------------- left(T_mcoaaE["eta_num"],2)
- care_site_source_value ------------ T_mcoaaE["eta_num"]

In [12]:
# Connexion à la base de données SQLite
conn = sqlite3.connect('health_data.db')
cursor = conn.cursor()

In [13]:
# Créer la table Care Site
cursor.execute('''
CREATE TABLE Care_Site (
    cc_site_id INTEGER PRIMARY KEY,
    care_site_name TEXT NOT NULL,
    location_id INTEGER NOT NULL,
    care_site_source_value TEXT  NOT NULL
)
''')

In [14]:
# Insérer les données à partir du DataFrame T_mcoaaE
care_site_data = [
    (index + 1, row['soc_rai'], int(str(row['eta_num'])[:2]), str(row['eta_num']))
    for index, row in T_mcoaaE.iterrows()
]

In [15]:
care_site_data

[(1, "l'Hôpital Privé des Peupliers", 75, '750300360'),
 (2, ' Pharmacie Plaisance', 75, '750023772')]

In [16]:
# Insérer les données
cursor.executemany('INSERT INTO Care_Site VALUES (?, ?, ?, ?)', care_site_data)

In [17]:
# Sauvegarder et fermer la connexion
conn.commit()
conn.close()

## 4.3 Table Provider

Correspondance des variables:
- provider_id ------------ index
- specialty_source_value ----------- label dans ir_act_v & ir_spe_v
- specialty_concept_id ----------- selon ATHENA:
                - id: 38004171	code: 247100000X	name: Radiologic Technologist (radiologue)
                - id: 38003731	code: 163WG0000X	name: Registered General Practice Nurse(médecin généraliste)
                - id: 38003810	code: 183500000X	name: Pharmacist (pharmacien)
                - id: 38004490	code: 65	        name: Physical Therapist (kinésitherapeute)
- provider_source_value ---------- pfs_act_nat dans ir_act_v & pfs_spe_cod dans ir_spe_v

In [18]:
ir_act = ir_act_v.rename(columns={"pfs_act_nat":"pfs"})
ir_spe = ir_spe_v.rename(columns={"pfs_spe_cod":"pfs"})
provider_data=pd.concat([ir_act,ir_spe],ignore_index=True)
provider_data=provider_data.sort_values(by="pfs",ignore_index=True)
provider_data["spe_con"]=[38003731,38004171,38004490,38003810]
provider_data=provider_data[["label","spe_con","pfs"]]

# Réinitialiser l'index pour convertir l'index en colonne
provider_data = provider_data.reset_index()

# Renommer la colonne d'index si nécessaire
provider_data = provider_data.rename(columns={'index': 'index_col'})

In [19]:
provider_data

,index_col,label,spe_con,pfs
0,0,Médecin généraliste,38003731,1
1,1,Radiologue,38004171,6
2,2,Kinésithérapeute,38004490,26
3,3,Pharmacien,38003810,50


In [20]:
# Initialiser une session Spark
spark = SparkSession.builder.appName("ProviderTable")\
    .config("spark.executor.memory", "1g") \
    .config("spark.driver.memory", "1g") \
    .config("spark.ui.showConsoleProgress", "true")\
    .master("local[*]") \
        .getOrCreate()

In [21]:
# Définir le schéma
schema = StructType([
    StructField("provider_id", IntegerType(), False),
    StructField("specialty_source_value", StringType(), False),
    StructField("specialty_concept_id", IntegerType(), False),
    StructField("provider_source_value", IntegerType(), False)
])

In [22]:
# Créer le DataFrame
provider_df = spark.createDataFrame(provider_data, schema)


In [23]:
# Sauvegarder en format Parquet
provider_df.write.parquet("Provider.parquet")

# 5. Validation des transformations

In [24]:
#  table Person
person_df = pd.read_csv('Person.csv')
print("\nPerson DataFrame:")
person_df.head()


Person DataFrame:


,person_id,gender_concept_id,year_of_birth,month_of_birth,person_source_value,location_id,gender_source_value
0,1,8532,1963,12,DPXX:00000000000000001X,75,2
1,2,8507,1971,2,DPXX:000000000000002X,93,1
2,3,8507,1962,12,DPXX:000000000000003X,93,1
3,4,8532,1959,3,DPXX:000000000000004X,94,2
4,5,8507,1998,4,DPXX:000000000000005X,93,1


In [25]:
# table Care Site
conn = sqlite3.connect('health_data.db')
care_site_df = pd.read_sql_query("SELECT * FROM Care_Site", conn)
print("\nCare Site DataFrame:")
care_site_df


Care Site DataFrame:


,cc_site_id,care_site_name,location_id,care_site_source_value
0,1,l'Hôpital Privé des Peupliers,75,750300360
1,2,Pharmacie Plaisance,75,750023772


In [26]:
conn.close()

In [27]:
# fichier Parquet
provider_df = spark.read.parquet("Provider.parquet")
print("\nProvider DataFrame:")
provider_df.show()


Provider DataFrame:
+-----------+----------------------+--------------------+---------------------+
|provider_id|specialty_source_value|specialty_concept_id|provider_source_value|
+-----------+----------------------+--------------------+---------------------+
|          0|   Médecin généraliste|            38003731|                    1|
|          1|            Radiologue|            38004171|                    6|
|          2|      Kinésithérapeute|            38004490|                   26|
|          3|            Pharmacien|            38003810|                   50|
+-----------+----------------------+--------------------+---------------------+



# 6. Annexes

## Documentation de référence:
 - **[OMOP Common Data Model](https://www.ohdsi.org/data-standardization/the-common-data-model/)**
 - **[SNDS Documentation](https://documentation-snds.health-data-hub.fr/)**